# NAME

> Fill in a module description here

In [1]:
# | default_exp examples


In [2]:
# | export
from __future__ import annotations
from typing import Tuple
import snowflake.snowpark.functions as F
from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.types import *

from snowflake_feature_store.connection import get_connection
from snowflake_feature_store.manager import feature_store_session
from snowflake_feature_store.feature_view import FeatureViewConfig
from snowflake_feature_store.transforms import fill_na, date_diff

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# | export
def generate_demo_data(
    session: Session, 
    schema: str,
    num_customers: int = 1000, 
    ltv_multiplier: float = 1.0, 
    session_length_multiplier: float = 1.0, 
    month: int = 5
) -> None:
    "Generate synthetic customer data for demonstration"
    # Create synthetic data columns
    random_id = F.uniform(0,5, F.random()).as_('RAND_ID')
    email = F.concat(
        F.call_builtin('RANDSTR', 10, F.random()), 
        F.lit('@'), 
        F.call_builtin('RANDSTR', 5, F.random()), 
        F.lit('.com')
    ).as_('EMAIL')
    
    # Customer attributes
    gender = F.when(F.uniform(1,10,F.random())<=7, F.lit('MALE')).otherwise('FEMALE').as_('GENDER')
    ltv = (F.round(F.uniform(100,75000,F.random()) / 100, 2) * ltv_multiplier).as_('LIFE_TIME_VALUE')
    month_col = (F.lit(month)).as_('MONTH')
    
    # Membership info
    membership_status = (
        F.when(F.col('LIFE_TIME_VALUE') < 150, F.lit('BASIC'))
        .when(F.col('LIFE_TIME_VALUE') < 250, F.lit('BRONZE'))
        .when(F.col('LIFE_TIME_VALUE') < 350, F.lit('SILVER'))
        .when(F.col('LIFE_TIME_VALUE') < 550, F.lit('GOLD'))
        .when(F.col('LIFE_TIME_VALUE') < 650, F.lit('PLATIN'))
        .otherwise('DIAMOND')
    ).as_('MEMBERSHIP_STATUS')
    
    membership_length = F.date_from_parts(
        2024, month, F.uniform(1,28,F.random(1))
    ).as_('MEMBER_JOIN_DATE')
    
    # Behavioral metrics
    avg_session_length = (
        F.col('LIFE_TIME_VALUE') / 100 + 
        F.uniform(0,5, F.random()) * session_length_multiplier
    ).cast(FloatType()).as_('AVG_SESSION_LENGTH_MIN')
    
    avg_time_on_app = (
        F.col('LIFE_TIME_VALUE') / 100 + 
        F.uniform(1,7, F.random())
    ).cast(FloatType()).as_('AVG_TIME_ON_APP_MIN')
    
    avg_time_on_website = (
        F.col('LIFE_TIME_VALUE') / 100 + 
        F.uniform(3,7, F.random())
    ).cast(FloatType()).as_('AVG_TIME_ON_WEBSITE_MIN')

    # Generate base DataFrame
    df = session.generator(
        random_id, email, month_col, ltv, gender,
        membership_status, membership_length, 
        avg_session_length, avg_time_on_app, avg_time_on_website, 
        rowcount=num_customers
    )

    # Add missing data patterns
    df = (df.with_column('AVG_SESSION_LENGTH_MIN', 
                        F.when(F.col('RAND_ID') == 2, None)
                        .otherwise(F.col('AVG_SESSION_LENGTH_MIN')))
          .with_column('AVG_TIME_ON_APP_MIN',
                      F.when(F.col('RAND_ID') == 3, None)
                      .otherwise(F.col('AVG_TIME_ON_APP_MIN')))
          .with_column('AVG_TIME_ON_WEBSITE_MIN',
                      F.when(F.col('RAND_ID') == 4, None)
                      .otherwise(F.col('AVG_TIME_ON_WEBSITE_MIN')))
          .drop('RAND_ID')
          .cache_result())

    for name, columns in {
        'CUSTOMER_LIFE_TIME_VALUE': ['EMAIL','LIFE_TIME_VALUE','MONTH'],
        'CUSTOMER_GENERAL_DATA': ['EMAIL','GENDER','MEMBERSHIP_STATUS','MEMBER_JOIN_DATE'],
        'CUSTOMER_BEHAVIOR_DATA': ['EMAIL','AVG_SESSION_LENGTH_MIN','AVG_TIME_ON_APP_MIN','AVG_TIME_ON_WEBSITE_MIN']
    }.items():
        full_table_name = f"{session.get_current_database()}.{schema}.{name}"
        df[columns].write.save_as_table(full_table_name, mode='overwrite')
        print(f'Added {num_customers} customers to table: {full_table_name}')


In [4]:
# | export
def get_example_data(
    session: Session,
    schema: str  # Add schema parameter
) -> Tuple[DataFrame, DataFrame, DataFrame]:
    """Load example customer data tables with correct types
    
    Args:
        session: Active Snowflake session
        schema: Schema where tables are located
        
    Returns:
        (ltv_df, cust_df, behavior_df) tuple of DataFrames
    """
    print('Loading example datasets...')
    generate_demo_data(session, schema, month=4)
    
    database = session.get_current_database()
    
    dfs = {
        'ltv': session.table(f"{database}.{schema}.CUSTOMER_LIFE_TIME_VALUE")
               .drop('MONTH')
               .select('EMAIL', 'LIFE_TIME_VALUE'),
               
        'behavior': session.table(f"{database}.{schema}.CUSTOMER_BEHAVIOR_DATA")
                   .select(
                       'EMAIL',
                       *[F.col(c).cast(DoubleType()).alias(c) for c in 
                         ['AVG_SESSION_LENGTH_MIN', 'AVG_TIME_ON_APP_MIN', 'AVG_TIME_ON_WEBSITE_MIN']]
                   ),
                   
        'customer': session.table(f"{database}.{schema}.CUSTOMER_GENERAL_DATA")
                   .select(
                       'EMAIL', 'GENDER', 'MEMBERSHIP_STATUS',
                       F.col('MEMBER_JOIN_DATE').cast(DateType()).alias('MEMBER_JOIN_DATE')
                   )
    }
    
    # Show samples and schemas
    for name, df in dfs.items():
        print(f'\n{name.title()} Data:')
        df.show(2)
        print(f'{name.title()} Schema:', df.schema)
    
    return dfs['ltv'], dfs['customer'], dfs['behavior']


In [5]:

# | export
def create_feature_configs() -> Tuple[FeatureViewConfig, FeatureViewConfig]:
    """Create example feature view configurations
    
    Returns:
        Tuple of (behavior_config, profile_config)
    """
    configs = {
        'behavior': FeatureViewConfig(
            name="customer_behavior",
            domain="RETAIL",
            entity="CUSTOMER",
            feature_type="BEHAVIOR",
            major_version=1,
            refresh_frequency="1 day",
            feature_descriptions={
                "AVG_SESSION_LENGTH_MIN": "Average session length in minutes",
                "AVG_TIME_ON_APP_MIN": "Average time on app per day",
                "AVG_TIME_ON_WEBSITE_MIN": "Average time on website per day"
            }
        ),
        
        'profile': FeatureViewConfig(
            name="customer_profile",
            domain="RETAIL",
            entity="CUSTOMER",
            feature_type="PROFILE",
            major_version=1,
            timestamp_col="MEMBER_JOIN_DATE",
            refresh_frequency="1 day",
            feature_descriptions={
                "MEMBERSHIP_LENGTH_DAYS": "Days since joining",
                "GENDER": "Customer gender",
                "MEMBERSHIP_STATUS": "Current membership level"
            }
        )
    }
    
    return configs['behavior'], configs['profile']


In [6]:
# | hide
def test_example_data():
    "Test example data generation and loading"
    conn = get_connection()
    session = conn.session
    schema = conn.schema
    
    ltv_df, cust_df, behavior_df = get_example_data(session, schema=schema)
    
    # Test data shapes
    assert ltv_df.count() > 0, "LTV data is empty"
    assert cust_df.count() > 0, "Customer data is empty"
    assert behavior_df.count() > 0, "Behavior data is empty"
    
    # Helper function to get column case-insensitively
    def get_column_type(df: DataFrame, col_name: str) -> type:
        # Find the actual column name (case-insensitive)
        actual_col = next(
            field.name for field in df.schema.fields 
            if field.name.upper() == col_name.upper()
        )
        return type(df.schema[actual_col].datatype)
    
    # Print schema for debugging
    print("\nActual schemas:")
    for name, df in [('LTV', ltv_df), ('Customer', cust_df), ('Behavior', behavior_df)]:
        print(f"\n{name} Schema:")
        for field in df.schema.fields:
            print(f"  {field.name}: {type(field.datatype).__name__}")
    
    # Test data types with case-insensitive column names
    try:
        assert isinstance(ltv_df.schema['LIFE_TIME_VALUE'].datatype, DoubleType), "Wrong LTV type"
        assert isinstance(cust_df.schema['MEMBER_JOIN_DATE'].datatype, DateType), "Wrong date type"
        assert isinstance(behavior_df.schema['AVG_SESSION_LENGTH_MIN'].datatype, DoubleType), "Wrong session length type"
        print("\nAll type checks passed!")
    except AssertionError as e:
        print(f"\nType check failed: {str(e)}")
        print(f"Expected types: DoubleType, DateType, DoubleType")
        print(f"Actual types:")
        print(f"  LTV: {type(ltv_df.schema['LIFE_TIME_VALUE'].datatype).__name__}")
        print(f"  Date: {type(cust_df.schema['MEMBER_JOIN_DATE'].datatype).__name__}")
        print(f"  Session: {type(behavior_df.schema['AVG_SESSION_LENGTH_MIN'].datatype).__name__}")
        raise

# Run tests
test_example_data()


Loading example datasets...
Added 1000 customers to table: "DATASCIENCE"."SIMPLE_ML_SCHEMA_2".CUSTOMER_LIFE_TIME_VALUE
Added 1000 customers to table: "DATASCIENCE"."SIMPLE_ML_SCHEMA_2".CUSTOMER_GENERAL_DATA
Added 1000 customers to table: "DATASCIENCE"."SIMPLE_ML_SCHEMA_2".CUSTOMER_BEHAVIOR_DATA

Ltv Data:
--------------------------------------------
|"EMAIL"               |"LIFE_TIME_VALUE"  |
--------------------------------------------
|kNdzn26u8R@LkHwu.com  |113.99             |
|OGpTifYvIl@O7joZ.com  |129.81             |
--------------------------------------------

Ltv Schema: StructType([StructField('EMAIL', StringType(16777216), nullable=False), StructField('LIFE_TIME_VALUE', DoubleType(), nullable=False)])

Behavior Data:
-------------------------------------------------------------------------------------------------------
|"EMAIL"               |"AVG_SESSION_LENGTH_MIN"  |"AVG_TIME_ON_APP_MIN"  |"AVG_TIME_ON_WEBSITE_MIN"  |
---------------------------------------------------

In [8]:
#| hide
import nbdev; nbdev.nbdev_export()